In [2]:
# sudo apt install tesseract-ocr
# sudo apt install libtesseract-dev

import os
os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata/'


In [3]:
pip install pdfminer.six pdfplumber pdf2image pytesseract pillow pdfminer PyPDF2 python-dateutil pytesseract

Note: you may need to restart the kernel to use updated packages.


In [4]:
import PyPDF2
# Для анализа структуры PDF и извлечения текста
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# Для извлечения текста из таблиц в PDF
import pdfplumber
# Для извлечения изображений из PDF
from PIL import Image
from pdf2image import convert_from_path
# Для выполнения OCR, чтобы извлекать тексты из изображений 
import pytesseract 
# Для удаления дополнительно созданных файлов
import os

import csv

In [5]:
# Создаём функцию для извлечения текста

def text_extraction(element):
    # Извлекаем текст из вложенного текстового элемента
    line_text = element.get_text()
    
    # Находим форматы текста
    # Инициализируем список со всеми форматами, встречающимися в строке текста
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Итеративно обходим каждый символ в строке текста
            for character in text_line:
                if isinstance(character, LTChar):
                    # Добавляем к символу название шрифта
                    line_formats.append(character.fontname)
                    # Добавляем к символу размер шрифта
                    line_formats.append(character.size)
    # Находим уникальные размеры и названия шрифтов в строке
    format_per_line = list(set(line_formats))
    
    # Возвращаем кортеж с текстом в каждой строке вместе с его форматом
    return (line_text, format_per_line)

In [6]:
# Создаём функцию для вырезания элементов изображений из PDF
def crop_image(element, pageObj):
    # Получаем координаты для вырезания изображения из PDF
    [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1] 
    # Обрезаем страницу по координатам (left, bottom, right, top)
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)
    # Сохраняем обрезанную страницу в новый PDF
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    # Сохраняем обрезанный PDF в новый файл
    with open('cropped_image.pdf', 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

# Создаём функцию для преобразования PDF в изображения
def convert_to_images(input_file,):
    images = convert_from_path(input_file)
    image = images[0]
    output_file = "PDF_image.png"
    image.save(output_file, "PNG")

# Создаём функцию для считывания текста из изображений
def image_to_text(image_path):
    # Считываем изображение
    img = Image.open(image_path)
    # Извлекаем текст из изображения
    text = pytesseract.image_to_string(img,lang='rus')
    return text

In [7]:
# Извлечение таблиц из страницы

def extract_table(pdf_path, page_num, table_num):
    # Открываем файл pdf
    pdf = pdfplumber.open(pdf_path)
    # Находим исследуемую страницу
    table_page = pdf.pages[page_num]
    # Извлекаем соответствующую таблицу
    table = table_page.extract_tables()[table_num]
    return table

# Преобразуем таблицу в соответствующий формат
def table_converter(table):
    table_string = ''
    # Итеративно обходим каждую строку в таблице
    for row_num in range(len(table)):
        row = table[row_num]
        # Удаляем разрыв строки из текста с переносом
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Преобразуем таблицу в строку
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Удаляем последний разрыв строки
    table_string = table_string[:-1]
    return table_string

In [8]:


# Находим путь к PDF
# pdf_path = '../../data/instrukciya-po-ehkspluatacii-v8.pdf'
# output_folder = '../../data/extracted_files/pdf/'
# Путь к папке


pdf_path = '../../data/doctorexplaindocument.pdf'

output_folder = '../../data/extracted_files/doctorexplaindocument/'

# создаём объект файла PDF
pdfFileObj = open(pdf_path, 'rb')
# создаём объект считывателя PDF
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Создаём словарь для извлечения текста из каждого изображения
text_per_page = {}
# Извлекаем страницы из PDF
for pagenum, page in enumerate(extract_pages(pdf_path)):
    
    # Инициализируем переменные, необходимые для извлечения текста со страницы
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Инициализируем количество исследованных таблиц
    table_num = 0
    first_element= True
    table_extraction_flag= False
    # Открываем файл pdf
    pdf = pdfplumber.open(pdf_path)
    # Находим исследуемую страницу
    page_tables = pdf.pages[pagenum]
    # Находим количество таблиц на странице
    tables = page_tables.find_tables()


    # Находим все элементы
    page_elements = [(element.y1, element) for element in page._objs]
    # Сортируем все элементы по порядку нахождения на странице
    page_elements.sort(key=lambda a: a[0], reverse=True)

    # Находим элементы, составляющие страницу
    for i,component in enumerate(page_elements):
        # Извлекаем положение верхнего края элемента в PDF
        pos= component[0]
        # Извлекаем элемент структуры страницы
        element = component[1]
        
        # Проверяем, является ли элемент текстовым
        if isinstance(element, LTTextContainer):
            # Проверяем, находится ли текст в таблице
            if table_extraction_flag == False:
                # Используем функцию извлечения текста и формата для каждого текстового элемента
                (line_text, format_per_line) = text_extraction(element)
                # Добавляем текст каждой строки к тексту страницы
                page_text.append(line_text)
                # Добавляем формат каждой строки, содержащей текст
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Пропускаем текст, находящийся в таблице
                pass

        # Проверяем элементы на наличие изображений
        if isinstance(element, LTFigure):
            # Вырезаем изображение из PDF
            crop_image(element, pageObj)
            # Преобразуем обрезанный pdf в изображение
            convert_to_images('cropped_image.pdf')
            # Извлекаем текст из изображения
            image_text = image_to_text('PDF_image.png')
            text_from_images.append(image_text)
            page_content.append(image_text)
            # Добавляем условное обозначение в списки текста и формата
            page_text.append('image')
            line_format.append('image')

        # Проверяем элементы на наличие таблиц
        if isinstance(element, LTRect):
            # Если первый прямоугольный элемент
            if first_element == True and (table_num+1) <= len(tables):
                # Находим ограничивающий прямоугольник таблицы
                lower_side = page.bbox[3] - tables[table_num].bbox[3]
                upper_side = element.y1 
                # Извлекаем информацию из таблицы
                table = extract_table(pdf_path, pagenum, table_num)
                # Преобразуем информацию таблицы в формат структурированной строки
                table_string = table_converter(table)
                # Добавляем строку таблицы в список
                text_from_tables.append(table_string)
                page_content.append(table_string)
                # Устанавливаем флаг True, чтобы избежать повторения содержимого
                table_extraction_flag = True
                # Преобразуем в другой элемент
                first_element = False
                # Добавляем условное обозначение в списки текста и формата
                page_text.append('table')
                line_format.append('table')

            # Проверяем, извлекли ли мы уже таблицы из этой страницы
            # if element.y0 >= lower_side and element.y1 <= upper_side:
            #     pass
            if i + 1 < len(page_elements):
              if not isinstance(page_elements[i+1][1], LTRect):
               table_extraction_flag = False
               first_element = True
               table_num += 1


    # Создаём ключ для словаря
    dctkey = 'Page_'+str(pagenum)
    # Добавляем список списков как значение ключа страницы
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

# Закрываем объект файла pdf
pdfFileObj.close()

# Удаляем созданные дополнительные файлы
os.remove('cropped_image.pdf')
os.remove('PDF_image.png')

# Удаляем содержимое страницы
result = ''.join(text_per_page['Page_0'][4])
print(result)

 
 
 
 
 
 
 
 
 
Инструкция по работе с системой 1С Экология
2023
 
 
 
 
 
 



In [9]:
# Путь к папке
output_folder = '../../data/extracted_files/'

# Создаем папку, если её нет
os.makedirs(output_folder, exist_ok=True)

# Сохраняем каждую страницу в отдельный файл
for page_num, page_content in text_per_page.items():
    page_text = page_content[4]  # Получаем содержимое страницы
    page_text_str = '\n'.join(page_text)  # Преобразуем список строк в одну строку
    page_file_path = os.path.join(output_folder, f'Page_{page_num}.txt')  # Путь к файлу страницы
    with open(page_file_path, 'w', encoding='utf-8') as page_file:
        page_file.write(page_text_str)  # Записываем содержимое страницы в файл
        print(f"Содержимое страницы {page_num} сохранено в файл {page_file_path}")

print("Извлеченные файлы сохранены.")


Содержимое страницы Page_0 сохранено в файл ../../data/extracted_files/Page_Page_0.txt
Содержимое страницы Page_1 сохранено в файл ../../data/extracted_files/Page_Page_1.txt
Содержимое страницы Page_2 сохранено в файл ../../data/extracted_files/Page_Page_2.txt
Содержимое страницы Page_3 сохранено в файл ../../data/extracted_files/Page_Page_3.txt
Содержимое страницы Page_4 сохранено в файл ../../data/extracted_files/Page_Page_4.txt
Содержимое страницы Page_5 сохранено в файл ../../data/extracted_files/Page_Page_5.txt
Содержимое страницы Page_6 сохранено в файл ../../data/extracted_files/Page_Page_6.txt
Содержимое страницы Page_7 сохранено в файл ../../data/extracted_files/Page_Page_7.txt
Содержимое страницы Page_8 сохранено в файл ../../data/extracted_files/Page_Page_8.txt
Содержимое страницы Page_9 сохранено в файл ../../data/extracted_files/Page_Page_9.txt
Содержимое страницы Page_10 сохранено в файл ../../data/extracted_files/Page_Page_10.txt
Содержимое страницы Page_11 сохранено в ф

In [10]:
csv_file_path = '../../data/extracted_files/extracted_data.csv'

# Открываем CSV-файл для записи
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Page', 'Text', 'Image Text', 'Table Text'])

    for page_num, page_content in text_per_page.items():
        page_text = ' '.join(page_content[0])  # Объединяем текст в одну строку
        image_text = ' '.join(page_content[2])  # Объединяем текст изображения в одну строку
        table_text = ' '.join(page_content[3])  # Объединяем текст таблицы в одну строку


        csv_writer.writerow([page_num, page_text, image_text, table_text])

print("Извлеченные данные сохранены в CSV-файл.")

Извлеченные данные сохранены в CSV-файл.
